In [2]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:5 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:9 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:11 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:12 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:13 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:14 http://security.ubuntu.com/ubuntu bionic-security/universe amd64 Packages [1,430

In [4]:
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Books_v1_02.tsv.gz"

from pyspark import SparkFiles
spark.sparkContext.addFile(url)
spark_df = spark.read.csv(SparkFiles.get("amazon_reviews_us_Books_v1_02.tsv.gz"), sep="\t", header=True)
spark_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   12076615| RQ58W7SMO911M|0385730586|     122662979|Sisterhood of the...|           Books|          4|            2|          3|   N|                N|this book was a g...|this boook was a ...| 2005-10-14|
|         US|   12703090|  RF6IUKMGL8SF|0811828964|      56191234|The Bad Girl's Gu...|           Books|          3|    

In [5]:
df_select = spark_df.select(["star_rating", "helpful_votes", "total_votes", "vine", "verified_purchase"])
df_select.show()

+-----------+-------------+-----------+----+-----------------+
|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+-----------+-------------+-----------+----+-----------------+
|          4|            2|          3|   N|                N|
|          3|            5|          5|   N|                N|
|          4|            1|         22|   N|                N|
|          5|            2|          2|   N|                N|
|          4|            0|          2|   N|                N|
|          4|            2|          2|   N|                N|
|          4|            9|         11|   N|                N|
|          4|            3|         10|   N|                N|
|          5|           16|         20|   N|                Y|
|          4|            0|          0|   N|                N|
|          5|            0|          0|   N|                N|
|          5|            1|          1|   N|                Y|
|          4|           14|         14|   N|           

In [6]:

# Drop NA and duplicates
df_select = df_select.dropna(how='any')
df_select.dropDuplicates()
df_select.show()

+-----------+-------------+-----------+----+-----------------+
|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+-----------+-------------+-----------+----+-----------------+
|          4|            2|          3|   N|                N|
|          3|            5|          5|   N|                N|
|          4|            1|         22|   N|                N|
|          5|            2|          2|   N|                N|
|          4|            0|          2|   N|                N|
|          4|            2|          2|   N|                N|
|          4|            9|         11|   N|                N|
|          4|            3|         10|   N|                N|
|          5|           16|         20|   N|                Y|
|          4|            0|          0|   N|                N|
|          5|            0|          0|   N|                N|
|          5|            1|          1|   N|                Y|
|          4|           14|         14|   N|           

In [7]:
df_vine = df_select.filter("total_votes>=20").filter(df_select["helpful_votes"]/df_select["total_votes"] >= 0.5)
df_vine.show()

+-----------+-------------+-----------+----+-----------------+
|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+-----------+-------------+-----------+----+-----------------+
|          5|           16|         20|   N|                Y|
|          5|           21|         31|   N|                N|
|          5|           27|         28|   N|                N|
|          4|           27|         27|   N|                N|
|          1|          268|        307|   N|                N|
|          4|           24|         27|   N|                Y|
|          5|           23|         24|   N|                N|
|          5|          145|        184|   N|                N|
|          5|           46|         47|   N|                N|
|          4|           28|         30|   N|                N|
|          2|           15|         24|   N|                N|
|          5|           26|         29|   N|                N|
|          4|           16|         21|   N|           

In [8]:
from pyspark.sql.functions import col, avg
df_paid = df_vine.filter("vine='Y'")
df_unpaid = df_vine.filter("vine='N'")

In [9]:

df_paid.describe().show()

+-------+-----------+-------------+-----------+----+-----------------+
|summary|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+-------+-----------+-------------+-----------+----+-----------------+
|  count|          0|            0|          0|   0|                0|
|   mean|       null|         null|       null|null|             null|
| stddev|       null|         null|       null|null|             null|
|    min|       null|         null|       null|null|             null|
|    max|       null|         null|       null|null|             null|
+-------+-----------+-------------+-----------+----+-----------------+



In [10]:
df_unpaid.describe().show()

+-------+------------------+----------------+-----------------+------+-----------------+
|summary|       star_rating|   helpful_votes|      total_votes|  vine|verified_purchase|
+-------+------------------+----------------+-----------------+------+-----------------+
|  count|            403807|          403807|           403807|403807|           403807|
|   mean|  4.07847560839708|41.5214273155245|47.58547028654778|  null|             null|
| stddev|1.3710414293829698|62.7084588623475|68.00152863386589|  null|             null|
|    min|                 1|              10|              100|     N|                N|
|    max|                 5|             998|              999|     N|                Y|
+-------+------------------+----------------+-----------------+------+-----------------+



In [11]:
five_star = df_paid[df_paid['star_rating'] == 5].count()
five_star

0

In [12]:

all_paid = df_paid.count()
all_paid

0

In [13]:
# Pecentage of five-star reviews in Vine
print(five_star/all_paid)

ZeroDivisionError: ignored

In [14]:
unpaid_five_star = df_unpaid[df_paid['star_rating'] == 5].count()
unpaid_five_star

242889

In [15]:
all_unpaid = df_unpaid.count()
all_unpaid

403807

In [16]:
# Pecentage of five-star reviews in non-Vine
print(unpaid_five_star/all_unpaid)

0.6014977452099641


**BONUS - Verified vs Unverified Reviews**

In [17]:
from pyspark.sql.functions import col, avg
df_verified = df_vine.filter("verified_purchase='Y'")
df_unverified = df_vine.filter("verified_purchase='N'")

In [18]:
df_verified.describe().show()

+-------+------------------+-----------------+-----------------+-----+-----------------+
|summary|       star_rating|    helpful_votes|      total_votes| vine|verified_purchase|
+-------+------------------+-----------------+-----------------+-----+-----------------+
|  count|             46200|            46200|            46200|46200|            46200|
|   mean|4.1301298701298705|46.13898268398268|51.77588744588745| null|             null|
| stddev|1.2793506195589872|69.23685683870448|75.27096840896195| null|             null|
|    min|                 1|               10|              100|    N|                Y|
|    max|                 5|               99|               99|    N|                Y|
+-------+------------------+-----------------+-----------------+-----+-----------------+



In [19]:
df_unverified.describe().show()

+-------+------------------+------------------+-----------------+------+-----------------+
|summary|       star_rating|     helpful_votes|      total_votes|  vine|verified_purchase|
+-------+------------------+------------------+-----------------+------+-----------------+
|  count|            357607|            357607|           357607|357607|           357607|
|   mean|  4.07180228574944|40.924875631629135|47.04410148570917|  null|             null|
| stddev|1.3823045433007342|  61.7897113695581|66.98586975848994|  null|             null|
|    min|                 1|                10|              100|     N|                N|
|    max|                 5|               998|              999|     N|                N|
+-------+------------------+------------------+-----------------+------+-----------------+



In [20]:
verified_five_star = df_verified[df_verified['star_rating'] == 5].count()
verified_five_star

27495

In [21]:
all_verified = df_verified.count()
all_verified

46200

In [22]:
# Percentage of five-star verified reviews in Vine
print(verified_five_star/all_verified)

0.5951298701298702


In [23]:
unverified_five_star = df_unverified[df_unverified['star_rating'] == 5].count()
unverified_five_star

215394

In [24]:
all_unverified = df_unverified.count()
all_unverified

357607

In [25]:
# Percentage of five-star verified reviews in Vine
print(unverified_five_star/all_unverified)

0.6023204243764804
